In [11]:
import pandas as pd
import torch
from transformers import (
    GPT2Tokenizer, GPT2Model, BertTokenizer, BertModel
)
# from pycaret.classification import *
import numpy as np

In [2]:
# Load the datasets
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
indicator_mapping = pd.read_csv('data/misconception_mapping.csv')
data_columns = [
    'QuestionId', 'ConstructId', 'ConstructName', 'SubjectId',
    'SubjectName', 'CorrectAnswer', 'QuestionText', 'AnswerAText',
    'AnswerBText', 'AnswerCText', 'AnswerDText', 'MisconceptionAId',
    'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId'
]


In [7]:
train_df

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864,1864,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,C,What is the range of the following numbers?\n\...,\( 5 \),\( 11 \),\( 23 \),\( 16 \),2456.0,691.0,NaN,1349.0
1865,1865,2695,"Describe an enlargement, with no centre of enl...",90,Length Scale Factors in Similar Shapes,B,Shape \( Q \) is an enlargement of shape \( P ...,\( 3 \div 11 \),\( 11 \div 3 \),\( 3 \times 11 \),\( 11-3 \),1500.0,NaN,2442.0,1258.0
1866,1866,854,Use the order of operations to carry out calcu...,33,BIDMAS,B,What does the following equal?\n\[\n8-7+10 \ti...,\( 36 \),\( 31 \),\( -29 \),\( 33 \),NaN,NaN,2306.0,1507.0
1867,1867,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,Tom and Katie are discussing congruence and si...,Only\nTom,Only Katie,Both Tom and Katie,Neither is correct,2312.0,NaN,2312.0,2312.0


#### GPT

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

input_ids = tokenizer("E = mc^2", return_tensors='pt').input_ids
outputs = model(input_ids)
embedding = outputs.last_hidden_state
embedding

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/Users/nafisgh/Documents/GitHub/Kaggle_Eedi/.venv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/nafisgh/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/nafisgh/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/nafisgh/Documents/GitHub/Kaggle_Eedi/.venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/nafisgh/Documents/GitHub/Kaggle_Eedi/.venv/lib/python3.9/site-packages/traitlets/config/applicatio

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: cannot import name 'NP_SUPPORTED_MODULES' from 'torch._dynamo.utils' (/Users/nafisgh/Documents/GitHub/Kaggle_Eedi/.venv/lib/python3.9/site-packages/torch/_dynamo/utils.py)

In [10]:
df = train_df.copy()
# Function to generate embeddings using GPT-2
def get_gpt2_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Generate embeddings 
for col in ["QuestionText", "AnswerAText", "AnswerAText", "AnswerAText", "AnswerAText"]:
    df[f'{col}_embedding'] = df[col].apply(lambda x: get_gpt2_embedding(x))

# Combine the embeddings (concatenate the two columns)
# df['combined_embedding'] = df.apply(lambda row: torch.cat((torch.tensor(row['col1_embedding']), 
#                                                            torch.tensor(row['col2_embedding'])), dim=-1).numpy(), axis=1)

NameError: name 'tokenizer' is not defined

In [ ]:
# Convert the combined embeddings into separate columns for PyCaret
combined_embeddings = np.vstack(df['combined_embedding'].values)
feature_columns = [f'feature_{i}' for i in range(combined_embeddings.shape[1])]

# Create a new dataframe with the embeddings and a target column
df_features = pd.DataFrame(combined_embeddings, columns=feature_columns)

# Add a target column (for the sake of example, we'll create one)
df_features['target'] = [0, 1, 0]  # Example binary classification labels


# Initialize PyCaret classification experiment
clf_setup = setup(data=df_features, target='target', silent=True, session_id=123)

# Compare different models and select the best one
best_model = compare_models()

# Train the best model
final_model = finalize_model(best_model)

# Make predictions (if you have a test set)
# predictions = predict_model(final_model, data=test_data)


#### MathBERT

In [ ]:

tokenizer = BertTokenizer.from_pretrained("tbs17/MathBERT")
model = BertModel.from_pretrained("tbs17/MathBERT")

inputs = tokenizer("x^2 + y^2 = z^2", return_tensors="pt")
outputs = model(**inputs)
embedding = outputs.last_hidden_state
